In [14]:
# Import packages
import pandas as pd
from scipy.stats import chi2_contingency

# Import data
data = pd.read_csv('candidates_original_preprocessed.csv')

# Chi-squared tests for Hired vs. Not hired

Chi-squared tests to test whether sex/age/region are related to being hired. 

In [16]:
# List of variables to analyze
variables = ['Sex', 'Age Range', 'Region']

# Initialize a dictionary to store results
chi2_results = {}

# Loop through each variable
for variable in variables:
    print(f"\n--- Analysis for {variable} ---")
    
    # Create the contingency table with totals
    contingency_table = pd.crosstab(
        data[variable],
        data['Candidate State'].apply(lambda x: 'Hired' if x == 'Hired' else 'Other'),
        margins=True  # Add totals for rows and columns
    )
    
    # Perform the Chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    # Store the results
    chi2_results[variable] = {
        'Contingency Table': contingency_table,
        'Chi2': chi2,
        'p-value': p,
        'Degrees of Freedom': dof,
        'Expected Frequencies': expected
    }
    
    # Display the contingency table
    print("Contingency Table:\n", contingency_table)
    
    # Display the Chi-square test results
    print("Chi-Square Test Statistic:", chi2)
    print("p-value:", p)
    print("Degrees of Freedom:", dof)
#     print("Expected Frequencies:\n", expected)
    
    # Determine significance
    alpha = 0.05  # Significance level
    if p < alpha:
        print(f"Conclusion: The variable '{variable}' and a person being hired or not, are likely to be dependent (there is a significant association).")
    else:
        print(f"Conclusion: The variable '{variable}' and a person being hired or not, are likely to be independent (no significant association).")



--- Analysis for Sex ---
Contingency Table:
 Candidate State  Hired  Other    All
Sex                                 
Female             130   2694   2824
Male               338   9070   9408
All                468  11764  12232
Chi-Square Test Statistic: 6.029950636346644
p-value: 0.19692267929101923
Degrees of Freedom: 4
Conclusion: The variable 'Sex' and a person being hired or not, are likely to be independent (no significant association).

--- Analysis for Age Range ---
Contingency Table:
 Candidate State  Hired  Other    All
Age Range                           
20 - 25 years       30   3284   3314
26 - 30 years      168   5550   5718
31 - 35 years       90   1021   1111
36 - 40 years       49    384    433
40 - 45 years       33    200    233
< 20 years          45    877    922
> 45 years          53    448    501
All                468  11764  12232
Chi-Square Test Statistic: 342.81644326571313
p-value: 1.3200582257954743e-64
Degrees of Freedom: 14
Conclusion: The variable 'A

For 'Region', the #observations is <5 in for some. This does not adhere with the rules of thumb of the Chi-squared test, this needs to be accounted for still. 

In [17]:
# Create a column 'Area' for North, Center and South Italy, based on the Classification of Territorial Units for Statistics (NUTS)
data['Area'] = data['Region'].apply(
    lambda region: 'North' if region in ['Piedmont', 'Lombardy', 'Emilia Romagna', 'Liguria', 'Trentino Alto Adige', 'Veneto', 'Aosta Valley', 'Friuli Venezia Giulia']
    else 'Center' if region in ['Lazio', 'Marche', 'Tuscany', 'Umbria']
    else 'South' if region in ['Abruzzo', 'Basilicata', 'Campania', 'Calabria', 'Puglia', 'Molise', 'Sicily', 'Sardinia']
    else 'Other'  # Could be overseas, foreign, or any other
)

In [18]:
# List of variables to analyze
variables = ['Area']

# Initialize a dictionary to store results
chi2_results = {}

# Loop through each variable
for variable in variables:
    print(f"\n--- Analysis for {variable} ---")
    
    # Create the contingency table with totals
    contingency_table = pd.crosstab(
        data[variable],
        data['Candidate State'].apply(lambda x: 'Hired' if x == 'Hired' else 'Other'),
        margins=True  # Add totals for rows and columns
    )
    
    # Perform the Chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    # Store the results
    chi2_results[variable] = {
        'Contingency Table': contingency_table,
        'Chi2': chi2,
        'p-value': p,
        'Degrees of Freedom': dof,
        'Expected Frequencies': expected
    }
    
    # Display the contingency table
    print("Contingency Table:\n", contingency_table)
    
    # Display the Chi-square test results
    print("Chi-Square Test Statistic:", chi2)
    print("p-value:", p)
    print("Degrees of Freedom:", dof)
#     print("Expected Frequencies:\n", expected)
    
    # Determine significance
    alpha = 0.05  # Significance level
    if p < alpha:
        print(f"Conclusion: The variable '{variable}' and a person being hired or not, are likely to be dependent (there is a significant association).")
    else:
        print(f"Conclusion: The variable '{variable}' and a person being hired or not, are likely to be independent (no significant association).")



--- Analysis for Area ---
Contingency Table:
 Candidate State  Hired  Other    All
Area                                
Center              80   1144   1224
North              232   5983   6215
Other               10    311    321
South              146   4326   4472
All                468  11764  12232
Chi-Square Test Statistic: 28.843791818215497
p-value: 0.00033774573761188875
Degrees of Freedom: 8
Conclusion: The variable 'Area' and a person being hired or not, are likely to be dependent (there is a significant association).


# Chi-squared tests for various candidates states

## Candidate state vs. Sex

In [19]:
# Order of selection
selection_order = ['Imported', 'In selection', 'First contact', 'QM', 'Vivier', 'Economic proposal', 'Hired']

# Initialize a dictionary
contingency_tables_sex = {}

# Loop through each state in the process order
for i, state in enumerate(selection_order):
    post_states = selection_order[i+1:]
    
    if post_states:   # Skips hired as it has no states after
        contingency_table_sex = pd.DataFrame({
            f'Post-{state}': data[data['Candidate State'].isin(post_states)].groupby('Sex').size(),
            state: data[data['Candidate State'] == state].groupby('Sex').size()
        })
        contingency_tables_sex[state] = contingency_table_sex

In [20]:
contingency_tables_sex

{'Imported':         Post-Imported  Imported
 Sex                            
 Female           1090      1734
 Male             4435      4973,
 'In selection':         Post-In selection  In selection
 Sex                                    
 Female                750           340
 Male                 2900          1535,
 'First contact':         Post-First contact  First contact
 Sex                                      
 Female                 175            575
 Male                   569           2331,
 'QM':         Post-QM   QM
 Sex                 
 Female      147   28
 Male        406  163,
 'Vivier':         Post-Vivier  Vivier
 Sex                        
 Female          144       3
 Male            389      17,
 'Economic proposal':         Post-Economic proposal  Economic proposal
 Sex                                              
 Female                     130                 14
 Male                       338                 51}

In [21]:

# Dictionary to store Chi-square test results
chi2_results = {}

# Perform Chi-square test for each contingency table
for state, table in contingency_tables_sex.items():
    # Drop rows with missing values, if any, to ensure valid input
    table = table.fillna(0).astype(int)
    
    # Perform the Chi-square test
    chi2, p, dof, expected = chi2_contingency(table)
    
    # Store the results
    chi2_results[state] = {
        'Chi2': chi2,
        'p-value': p,
        'Degrees of Freedom': dof,
        'Expected Frequencies': expected
    }
    
    # Print results for the current state
    print(f"\n--- Analysis for {state} ---")
    print(f"Chi-Square Test Statistic: {chi2}")
    print(f"p-value: {p}")
    print(f"Degrees of Freedom: {dof}")
#     print(f"Expected Frequencies:\n{expected}")
    
    alpha = 0.05
    if p < alpha:
        print(f"Conclusion: For '{state}', the variables 'Sex' and 'Candidate State' are likely to be dependent (significant association).")
    else:
        print(f"Conclusion: For '{state}', the variables 'Sex' and 'Candidate State' are likely to be independent (no significant association).")


--- Analysis for Imported ---
Chi-Square Test Statistic: 63.66134178926101
p-value: 1.477558472913691e-15
Degrees of Freedom: 1
Conclusion: For 'Imported', the variables 'Sex' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for In selection ---
Chi-Square Test Statistic: 4.409181101166797
p-value: 0.03574599691749415
Degrees of Freedom: 1
Conclusion: For 'In selection', the variables 'Sex' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for First contact ---
Chi-Square Test Statistic: 4.834975066237263
p-value: 0.02788804410371374
Degrees of Freedom: 1
Conclusion: For 'First contact', the variables 'Sex' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for QM ---
Chi-Square Test Statistic: 10.565197261191749
p-value: 0.0011523673663921489
Degrees of Freedom: 1
Conclusion: For 'QM', the variables 'Sex' and 'Candidate State' are likely to be dependent (significant

## Candidate state vs. Age Range

In [22]:
# Order of selection
selection_order = ['Imported', 'In selection', 'First contact', 'QM', 'Vivier', 'Economic proposal', 'Hired']

# Initialize a dictionary
contingency_tables_age = {}

# Loop through each state in the process order
for i, state in enumerate(selection_order):
    post_states = selection_order[i+1:]
    
    if post_states:   # Skips hired as it has no states after
        contingency_table_age = pd.DataFrame({
            f'Post-{state}': data[data['Candidate State'].isin(post_states)].groupby('Age Range').size(),
            state: data[data['Candidate State'] == state].groupby('Age Range').size()
        })
        contingency_tables_age[state] = contingency_table_age

In [23]:
contingency_tables_age

{'Imported':                Post-Imported  Imported
 Age Range                             
 20 - 25 years            602      2712
 26 - 30 years           2229      3489
 31 - 35 years            739       372
 36 - 40 years            371        62
 40 - 45 years            223        10
 < 20 years               878        44
 > 45 years               483        18,
 'In selection':                Post-In selection  In selection
 Age Range                                     
 20 - 25 years                415           187
 26 - 30 years               1489           740
 31 - 35 years                483           256
 36 - 40 years                251           120
 40 - 45 years                158            65
 < 20 years                   534           344
 > 45 years                   320           163,
 'First contact':                Post-First contact  First contact
 Age Range                                       
 20 - 25 years                  42            373
 26 - 30 ye

In [24]:

# Dictionary to store Chi-square test results
chi2_results = {}

# Perform Chi-square test for each contingency table
for state, table in contingency_tables_age.items():
    # Drop rows with missing values, if any, to ensure valid input
    table = table.fillna(0).astype(int)
    
    # Perform the Chi-square test
    chi2, p, dof, expected = chi2_contingency(table)
    
    # Store the results
    chi2_results[state] = {
        'Chi2': chi2,
        'p-value': p,
        'Degrees of Freedom': dof,
        'Expected Frequencies': expected
    }
    
    # Print results for the current state
    print(f"\n--- Analysis for {state} ---")
    print(f"Chi-Square Test Statistic: {chi2}")
    print(f"p-value: {p}")
    print(f"Degrees of Freedom: {dof}")
#     print(f"Expected Frequencies:\n{expected}")
    
    alpha = 0.05
    if p < alpha:
        print(f"Conclusion: For '{state}', the variables 'Age Range' and 'Candidate State' are likely to be dependent (significant association).")
    else:
        print(f"Conclusion: For '{state}', the variables 'Age Range' and 'Candidate State' are likely to be independent (no significant association).")


--- Analysis for Imported ---
Chi-Square Test Statistic: 3259.7363634241924
p-value: 0.0
Degrees of Freedom: 6
Conclusion: For 'Imported', the variables 'Age Range' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for In selection ---
Chi-Square Test Statistic: 16.396566428031583
p-value: 0.011776528708639615
Degrees of Freedom: 6
Conclusion: For 'In selection', the variables 'Age Range' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for First contact ---
Chi-Square Test Statistic: 92.78203768768347
p-value: 8.001878080863548e-18
Degrees of Freedom: 6
Conclusion: For 'First contact', the variables 'Age Range' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for QM ---
Chi-Square Test Statistic: 15.833494766518513
p-value: 0.014676125266923598
Degrees of Freedom: 6
Conclusion: For 'QM', the variables 'Age Range' and 'Candidate State' are likely to be dependent (s

# Candidate state vs. Region

In [25]:
# Order of selection
selection_order = ['Imported', 'In selection', 'First contact', 'QM', 'Vivier', 'Economic proposal', 'Hired']

# Initialize a dictionary
contingency_tables_area = {}

# Loop through each state in the process order
for i, state in enumerate(selection_order):
    post_states = selection_order[i+1:]
    
    if post_states:   # Skips hired as it has no states after
        contingency_table_area = pd.DataFrame({
            f'Post-{state}': data[data['Candidate State'].isin(post_states)].groupby('Area').size(),
            state: data[data['Candidate State'] == state].groupby('Area').size()
        })
        contingency_tables_area[state] = contingency_table_area

In [26]:
contingency_tables_area

{'Imported':         Post-Imported  Imported
 Area                           
 Center            872       352
 North            2818      3397
 Other             270        51
 South            1565      2907,
 'In selection':         Post-In selection  In selection
 Area                                   
 Center                534           338
 North                1877           941
 Other                 203            67
 South                1036           529,
 'First contact':         Post-First contact  First contact
 Area                                     
 Center                 132            402
 North                  346           1531
 Other                   31            172
 South                  235            801,
 'QM':         Post-QM  QM
 Area               
 Center       94  38
 North       263  83
 Other        18  13
 South       178  57,
 'Vivier':         Post-Vivier  Vivier
 Area                       
 Center           90       4
 North           257

In [27]:

# Dictionary to store Chi-square test results
chi2_results = {}

# Perform Chi-square test for each contingency table
for state, table in contingency_tables_area.items():
    # Drop rows with missing values, if any, to ensure valid input
    table = table.fillna(0).astype(int)
    
    # Perform the Chi-square test
    chi2, p, dof, expected = chi2_contingency(table)
    
    # Store the results
    chi2_results[state] = {
        'Chi2': chi2,
        'p-value': p,
        'Degrees of Freedom': dof,
        'Expected Frequencies': expected
    }
    
    # Print results for the current state
    print(f"\n--- Analysis for {state} ---")
    print(f"Chi-Square Test Statistic: {chi2}")
    print(f"p-value: {p}")
    print(f"Degrees of Freedom: {dof}")
#     print(f"Expected Frequencies:\n{expected}")
    
    alpha = 0.05
    if p < alpha:
        print(f"Conclusion: For '{state}', the variables 'Area' and 'Candidate State' are likely to be dependent (significant association).")
    else:
        print(f"Conclusion: For '{state}', the variables 'Area' and 'Candidate State' are likely to be independent (no significant association).")


--- Analysis for Imported ---
Chi-Square Test Statistic: 719.4864909071396
p-value: 1.2489565930501113e-155
Degrees of Freedom: 3
Conclusion: For 'Imported', the variables 'Area' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for In selection ---
Chi-Square Test Statistic: 19.45976522932024
p-value: 0.0002196248756575271
Degrees of Freedom: 3
Conclusion: For 'In selection', the variables 'Area' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for First contact ---
Chi-Square Test Statistic: 17.228713635120734
p-value: 0.0006341727118047743
Degrees of Freedom: 3
Conclusion: For 'First contact', the variables 'Area' and 'Candidate State' are likely to be dependent (significant association).

--- Analysis for QM ---
Chi-Square Test Statistic: 5.729847966664108
p-value: 0.12551952817761006
Degrees of Freedom: 3
Conclusion: For 'QM', the variables 'Area' and 'Candidate State' are likely to be independent (n

For both 'Age Range' and Area' in some cells in the contingency table for various candidate states, the #observations is <5. This does not adhere with the rules of thumb of the Chi-squared test.